In [18]:
!nvidia-smi
from tensorflow.keras import layers
import tensorflow as tf

Sat Mar  2 12:14:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.58                 Driver Version: 537.58       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   40C    P8               8W /  80W |   1846MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [19]:
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

In [20]:
#!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

In [21]:
#unzip_data('nlp_getting_started.zip')

In [22]:
import pandas as pd
train_df = pd.read_csv("train.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [23]:
train_df_shuffled = train_df.sample(frac=1, random_state = 42)

In [24]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [25]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [26]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [27]:
len(train_df),len(test_df)

(7613, 3263)

In [28]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target : {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text : \n{text}\n")
    print("----\n")
    


Target : 1 (real disaster)
Text : 
Photo: mothernaturenetwork: What is thundersnow? Hearing thunder during a snowstorm is extremely uncommon.... http://t.co/eYdAPauPvG

----

Target : 0 (not real disaster)
Text : 
To Supply and Install New FRP Emergency Slide  at Tunas KijangBank Negara Malaysia [Closing Date: 2015-08-14]... http://t.co/ZpqwKHFhNf

----

Target : 0 (not real disaster)
Text : 
@stvmlly Sorry for screaming at you and @Safferoonicle from the car I just kinda 'I KNOW THOSE PEOPLE LET'S YELL!' :/

----

Target : 1 (real disaster)
Text : 
Three beached whales rescued in Kerry - http://t.co/rQbsPUCjDF

----

Target : 0 (not real disaster)
Text : 
@supernovalester I feel so bad for them. I can literally feel that feeling of your heart sinking bc you didn't get anyone ugh jfc

----



In [29]:
#007 Splitting data into training and validation sets.mp4
from sklearn.model_selection import train_test_split
train_sentences , val_sentences ,train_labels,val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)
                                                                            
                                                                            

In [30]:
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [31]:
#008 Converting text data to numbers using tokenisation and embeddings (overview).mp4
#009 Setting up a TensorFlow TextVectorization layer to convert text to numbers.mp4
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#OOV  = "Out of Vocabulary"
text_vectorizer = TextVectorization(max_tokens=10000,
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)
                                    
                                    
                                    

In [32]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [33]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length,
                                   )

In [34]:
#010 Mapping the TextVectorization layer to text data and turning it into numbers
# Fit the text vectorizer to training text
text_vectorizer.adapt(train_sentences)

In [36]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 #novalismi Joseph Mallord William Turner - Shade and Darkness - the Evening of the Deluge 1843 (Clicca sul titolo... http://t.co/458DtR3ulx\n
Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    1,    1, 4247, 7301, 8615,    7, 3949,    2, 1415,    6,
           2,  400,    1,    1]], dtype=int64)>

In [39]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
#[UNK] meand unknown token

In [41]:
len(words_in_vocab)

10000

In [42]:
#011 Creating an Embedding layer to turn tokenised text into embedding vectors
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length = max_length
                             )

In [45]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Tried orange aftershock today. My life will never be the same\n
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01805753, -0.00197964, -0.0121241 , ..., -0.00976595,
          0.03103986,  0.03660316],
        [-0.02087772,  0.04928492,  0.04040204, ...,  0.02458907,
         -0.00144137, -0.01632774],
        [ 0.02997043, -0.04364026, -0.00178174, ...,  0.0095721 ,
          0.04786888, -0.04646401],
        ...,
        [-0.03660678, -0.02634165,  0.02119039, ...,  0.02451858,
          0.00231373, -0.03791336],
        [-0.03660678, -0.02634165,  0.02119039, ...,  0.02451858,
          0.00231373, -0.03791336],
        [-0.03660678, -0.02634165,  0.02119039, ...,  0.02451858,
          0.00231373, -0.03791336]]], dtype=float32)>

In [47]:
#check out a single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.01805753, -0.00197964, -0.0121241 , -0.04742119,  0.04738407,
        -0.01366136,  0.04001958,  0.04975906, -0.00747583, -0.02109395,
         0.03919418, -0.00070174,  0.00450822,  0.02884066, -0.00747037,
         0.00945855,  0.03971129,  0.04812605, -0.04077337,  0.01290239,
        -0.00816429, -0.02901891, -0.04223375, -0.02009356,  0.03504057,
        -0.02033211, -0.01512728, -0.00927365,  0.03235419,  0.00073852,
        -0.00292291, -0.00845692, -0.03789844, -0.01028174,  0.02496463,
         0.01369152, -0.01252797, -0.00936317,  0.0220147 ,  0.02961273,
         0.04792141,  0.01948626, -0.04662074,  0.02606816, -0.00901186,
        -0.02518814, -0.00018945, -0.0386846 ,  0.00891273, -0.00530674,
        -0.0032774 ,  0.01546619, -0.03334244,  0.01026727,  0.04879082,
         0.03210813, -0.01604513,  0.0010353 , -0.01952337, -0.02237332,
        -0.02157202,  0.01869443,  0.04874082, -0.01764309, -0.01312759,
  

In [48]:
#012 Discussing the various modelling experiments we're going to run
#013 Model 0 Building a baseline model to try and improve upon
#Multinomial naive bais using TF_IDF formula
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB()),

])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [50]:
#Evalute our baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achives an accuracy of : {baseline_score*100:.2f}%")

Our baseline model achives an accuracy of : 79.27%


In [51]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [52]:
#014 Creating a function to track and evaluate our model's results
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
    model_accuracy = accuracy_score(y_true,y_pred)*100
    model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true, y_pred,average="weighted")
    model_results = {"accuracy":model_accuracy,
                     "precision":model_precision,
                     "recall":model_recall,
                     "f1":model_f1}
    return model_results

In [53]:
baseline_results = calculate_results(y_true=val_labels,y_pred=baseline_preds)
baseline_results
#higher precision leads to less false positives
#higher recall leads to less false negetives

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [73]:
#015 Model 1 Building, fitting and evaluating our first deep model on text data
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers
SAVE_DIR = "model_logs"
#build model with Function API
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x) #layers.Reshape((1920,))(x) #.
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")
model_1.summary()



Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_3   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [74]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240302-202159
Epoch 1/5
215/215 [==============================] - 2s 5ms/step - loss: 0.4134 - accuracy: 0.9186 - val_loss: 0.4741 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2350 - accuracy: 0.9380 - val_loss: 0.4720 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1782 - accuracy: 0.9501 - val_loss: 0.4965 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1475 - accuracy: 0.9569 - val_loss: 0.5259 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1273 - accuracy: 0.9596 - val_loss: 0.5563 - val_accuracy: 0.7795


In [75]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.5563 - accuracy: 0.7795


[0.5563217401504517, 0.7795275449752808]

In [78]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0]

24/24 [==============================] - 0s 2ms/step


array([0.46392477], dtype=float32)

In [79]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [80]:
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [82]:
#calculate model 1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 77.95275590551181,
 'precision': 0.7813761276529175,
 'recall': 0.7795275590551181,
 'f1': 0.7775281328700309}

In [84]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [ ]:
#016 Visualising our model's learned word embeddings with TensorFlow's projector tool
